# Colab × GitHub (шаблон с SSH-ключами)

Цель: работа с GitHub через SSH. Ключи храним в Google Drive, чтобы сохранялись между сессиями.

Алгоритм:
1. Смонтировать Drive.
2. Сгенерировать ключ (или использовать существующий) в `MyDrive/.ssh`.
3. Добавить публичный ключ в GitHub → Settings → SSH and GPG keys.
4. Клонировать репозиторий по SSH и работать как обычно.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import subprocess, pathlib

def sh(cmd, cwd=None, check=True):
    return subprocess.run(cmd, cwd=cwd, shell=True, text=True, capture_output=True, check=check)

def git_config(user_name: str, user_email: str):
    sh(f'git config --global user.name "{user_name}"')
    sh(f'git config --global user.email "{user_email}"')
    sh('git config --global credential.helper ""')

git_config("SIA86", "sfron4ik@gmail.com")

In [ ]:
from pathlib import Path

DRIVE_SSH_DIR = Path('/content/drive/MyDrive/.ssh')
DRIVE_SSH_DIR.mkdir(parents=True, exist_ok=True)
ID_KEY = DRIVE_SSH_DIR / 'id_ed25519'
ID_PUB = DRIVE_SSH_DIR / 'id_ed25519.pub'

if not ID_KEY.exists():
    sh(f'ssh-keygen -t ed25519 -N "" -f "{ID_KEY}"')
    print("Сгенерирован новый ключ.")
else:
    print("Найден существующий ключ.")

sh('mkdir -p ~/.ssh && chmod 700 ~/.ssh')
sh(f'ln -sf "{ID_KEY}" ~/.ssh/id_ed25519')
sh(f'ln -sf "{ID_PUB}" ~/.ssh/id_ed25519.pub')
sh('chmod 600 ~/.ssh/id_ed25519 && chmod 644 ~/.ssh/id_ed25519.pub')
sh('ssh-keyscan github.com >> ~/.ssh/known_hosts || true')
sh('chmod 644 ~/.ssh/known_hosts')

print("Публичный ключ для GitHub:")
print(ID_PUB.read_text())

Найден существующий ключ.
Публичный ключ для GitHub:
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKot/XiGZ9/rvjmmZsCuGn1O0CIQzGUeiYLjyWCpdWN8 root@01bff0625f36



In [ ]:
res = sh('ssh -o StrictHostKeyChecking=yes -T git@github.com', check=False)
print(res.stdout or res.stderr)

Hi SIA86! You've successfully authenticated, but GitHub does not provide shell access.



In [ ]:
OWNER_AND_REPO = "SIA86/pretrained_rl_ppo_model"  # например: octocat/Hello-World
DEST_DIR = "/content/drive/MyDrive/stocks/pretrained_rl_ppo"
ssh_url = f"git@github.com:{OWNER_AND_REPO}.git"
sh(f'git clone "{ssh_url}" "{DEST_DIR}"')

CompletedProcess(args='git clone "git@github.com:SIA86/pretrained_rl_ppo_model.git" "/content/drive/MyDrive/stocks/pretrained_rl_ppo"', returncode=0, stdout='', stderr="Cloning into '/content/drive/MyDrive/stocks/pretrained_rl_ppo'...\n")

In [ ]:
# @title API
def git_status(cwd):
    return sh("git status -sb", cwd=cwd).stdout

def git_checkout(cwd, branch: str, create: bool=False):
    if create:
        sh(f'git checkout -b "{branch}"', cwd=cwd)
    else:
        sh(f'git checkout "{branch}"', cwd=cwd)

def git_pull(cwd, rebase=False):
    sh("git fetch --prune", cwd=cwd)
    sh("git pull --rebase" if rebase else "git pull", cwd=cwd)

def git_add_all(cwd):
    sh("git add -all", cwd=cwd)

def git_commit(cwd, message: str):
    sh(f'git commit -m "{message}"', cwd=cwd)

def git_push(cwd, remote="origin", branch="HEAD", set_upstream=False, force=False):
    flags = []
    if set_upstream: flags.append("-u")
    if force: flags.append("--force-with-lease")
    sh(f'git push {" ".join(flags)} {remote} {branch}', cwd=cwd)

In [ ]:
# проверка статуса
git_status(DEST_DIR)

In [15]:
# закоммитить изменения
git_add_all(DEST_DIR)
git_commit(DEST_DIR, "feat: Добавил блокнот для работы с гитхабом")
git_push(DEST_DIR, branch="main")